# import packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import gzip
import shutil
import os

C:\Users\monkeydc\.conda\envs\561\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# data processing

In [2]:
with gzip.open('glove.6B.100d.gz', 'rb') as f_in:
    with open('glove.6B.100d', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [3]:
def word_type(word:str):
    if word.isdigit(): #Is a digit
        return "unk_num"
    elif word.islower(): 
        return "unk_all_lower"    
    elif word.isupper(): 
        return "unk_all_upper"              
    elif any(char.isdigit() for char in word):
        return "unk_contain_num"    
    else:
        return "unk"

In [4]:
def read_in_data(filename):
    vocab = dict()
    sentences =[] # generating original output
    tags = set()
    tmp_sentence = []
    targets = []
    tmp_target = []
    with open(filename, "r") as file:
        for line in file.readlines():
            if len(line) > 1:
                _, word, tag = line.strip().split(" ")
                if word not in vocab.keys():
                    vocab[word] = 1
                else:
                    vocab[word] += 1
                tags.add(tag)
                tmp_sentence.append(word)
                tmp_target.append(tag)
            else:
                sentences.append(tmp_sentence)
                targets.append(tmp_target)
                tmp_sentence = []
                tmp_target = []
        if len(tmp_sentence)>0:
            sentences.append(tmp_sentence)
            targets.append(tmp_target)
    return vocab, sentences, tags, targets

def remove_low_frequency_word(vocab, occurences):
    candidates = set()
    for word in vocab.keys():
        if vocab[word] >= occurences:
            if any(w.isdigit() for w in word):
                candidates.add(word_type(word))
            else:
                candidates.add(word)
        else:
            candidates.add(word_type(word))
    return candidates


In [5]:
vocab_OCC, sentences, tags, targets = read_in_data("./data/train")

In [6]:
vocab = remove_low_frequency_word(vocab_OCC, occurences = 2)

In [7]:
def get_idx_vocab_tags(vocab, tags):
    word_to_idx = {}
    tag_to_idx = {}
    # word_to_idx["unk"] = 1
    word_to_idx["PAD"] = 0
    start_word = 1
    for word in vocab:
        word_to_idx[word] = start_word
        start_word += 1
    start_tag = 0
    for tag in tags:
        tag_to_idx[tag] = start_tag
        start_tag += 1
    return word_to_idx, tag_to_idx

In [8]:
word_to_idx, tag_to_idx = get_idx_vocab_tags(vocab, tags)

In [9]:
len(word_to_idx)

10732

In [10]:
print(word_to_idx)

{'PAD': 0, 'boosted': 1, 'previous': 2, 'unnamed': 3, 'estimated': 4, 'Skopje': 5, 'Fernando': 6, 'Pudvah': 7, 'Wickman': 8, 'Antwerp': 9, 'velodrome': 10, 'Niugini': 11, 'totalled': 12, 'vicious': 13, 'dissidents': 14, 'PRECIOUS': 15, 'complaining': 16, 'Rockies': 17, 'lingers': 18, 'easier': 19, 'Unsecured': 20, 'ABIDJAN': 21, 'Dick': 22, 'fill': 23, 'Yingkou': 24, 'Sukarnoputri': 25, 'Refined': 26, 'century': 27, 'cease': 28, 'troubled': 29, 'balloting': 30, 'Olaf': 31, 'carries': 32, 'Petr': 33, 'Roger': 34, 'STRAIGHT': 35, 'officials': 36, 'swings': 37, 'Eksportfinans': 38, 'complaints': 39, 'Congressman': 40, 'tool': 41, 'valuing': 42, 'failed': 43, 'Conservative': 44, 'objective': 45, 'bowled': 46, 'understand': 47, 'repeatedly': 48, 'defensive': 49, 'rebellion': 50, 'Rad': 51, 'past': 52, 'Mustapha': 53, 'fund': 54, 'Andria': 55, 'generated': 56, 'pilot': 57, 'GETS': 58, 'regime': 59, 'topple': 60, 'K': 61, 'immediate': 62, 'association': 63, 'McGrath': 64, 'Cologne': 65, 'TUE'

In [11]:
def get_vocab_tags_idx(word_to_idx, tag_to_idx):
    idx_to_word, idx_to_tag = {}, {}
    for word, idx in word_to_idx.items():
        idx_to_word[idx] = word
    for tag, idx in tag_to_idx.items():
        idx_to_tag[idx] = tag
    return idx_to_word, idx_to_tag

In [12]:
idx_to_word, idx_to_tag = get_vocab_tags_idx(word_to_idx, tag_to_idx)

In [13]:
def process_sentences(sentences:list, word_to_idx:dict):
    data = []
    for sentence in sentences:
        tmp_sentence = []
        for word in sentence:
            if word in word_to_idx.keys():
                tmp_sentence.append(word_to_idx[word])
            else:
                tmp_sentence.append(word_to_idx[word_type(word)])
        data.append(tmp_sentence)
    return data

def process_targets(targets:list, tag_to_idx:dict):
    data = []
    for target in targets:
        tmp_target = []
        for tag in target:
            tmp_target.append(tag_to_idx[tag])
        data.append(tmp_target)
    return data
    

In [14]:
data_X = process_sentences(sentences, word_to_idx)

In [15]:
data_y = process_targets(targets, tag_to_idx)

In [16]:
# get the maximum length of sentence in train
maximum_length = 0
for sample in data_X:
    if len(sample) > maximum_length:
        maximum_length = len(sample)

In [17]:
maximum_length

113

In [18]:
def padding_sentence(data: list, maximum_length=125):
    for i in range(len(data)):
        if len(data[i]) > maximum_length:  # Truncating
            data[i] = data[i][:maximum_length]
        elif len(data[i]) < maximum_length:  # Padding 
            data[i] = data[i] + [0] * (maximum_length - len(data[i]))
    return data

def padding_tags(data: list, maximum_length=125):
    for i in range(len(data)):
        if len(data[i]) > maximum_length:  # Truncating
            data[i] = data[i][:maximum_length]
        elif len(data[i]) < maximum_length:  # Padding with penalty score
            data[i] = data[i] + [-100] * (maximum_length - len(data[i]))
    return data

In [19]:
data_X = padding_sentence(data_X)
data_y = padding_tags(data_y)

In [20]:
X_train = torch.LongTensor(data_X)
Y_train = torch.LongTensor(data_y)

In [21]:
print(X_train)

tensor([[ 8325,  7832,   438,  ...,     0,     0,     0],
        [ 4318,  5931,     0,  ...,     0,     0,     0],
        [ 3658, 10239,     0,  ...,     0,     0,     0],
        ...,
        [10580, 10082,     0,  ...,     0,     0,     0],
        [ 1326,  6657,  6011,  ...,     0,     0,     0],
        [ 9544,     0,     0,  ...,     0,     0,     0]])


In [22]:
ds_train = TensorDataset(X_train, Y_train)
loader_train = DataLoader(ds_train, batch_size=16, shuffle=False)

In [23]:
embedding_weight = dict()
f = open(os.path.join('glove.6B.100d'), encoding='utf-8')
for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_emb = np.asarray(word_vector[1:], dtype='float32') 
    embedding_weight[word] = word_emb

embedding_dim = 100
embedding_input = np.zeros((len(word_to_idx), embedding_dim))

In [24]:
for word, idx in word_to_idx.items():
    embedding_vec = embedding_weight.get(word.lower())
    if embedding_vec is not None:
        embedding_input[idx] = embedding_vec

In [29]:
print(embedding_input)

[[-0.93299001 -0.14930999 -0.35146999 ...  0.40952    -0.016286
   0.22317   ]
 [ 0.024129    0.19566999 -0.15423    ...  0.17268001  0.35971999
   0.029541  ]
 [ 0.12222     0.13160001  0.27070001 ...  0.35692     0.64782
  -0.47792   ]
 ...
 [ 0.57792002  0.145      -0.096722   ...  0.1963      0.066402
   0.34397   ]
 [ 0.29019001  0.80497003  0.31187001 ... -0.33603001  0.45998001
  -0.11278   ]
 [ 0.20954999  0.0028596  -0.43630999 ...  0.31975999 -0.31375
   0.19766   ]]


In [30]:
#embedding_matrix_model = torch.LongTensor(embedding_matrix)
embedding_matrix_model = torch.Tensor(embedding_input)

# model structure define

In [31]:
class BiLSTMNER(nn.Module):
    def __init__(self, vocab_size, target_size, embedding_dim=100, lstm_hidden_dim=256, lstm_layers=1, lstm_dropout=0.33, linear_dim=128):
        super().__init__()        
        self.dropout = nn.Dropout(0.33)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden_dim,num_layers=lstm_layers,batch_first=True, bidirectional=True) #dropout=lstm_dropout)
        self.linear = nn.Linear(lstm_hidden_dim * 2, linear_dim)
        self.elu = nn.ELU()
        self.classifier = nn.Linear(linear_dim, target_size)

    def forward(self, sentence):
        embeds = self.dropout(self.embedding(sentence))
        lstm_out,_ = self.lstm(embeds)
        lstm_out = self.dropout(lstm_out)
        linear_out = self.linear(lstm_out)
        elu_out = self.elu(linear_out)
        output = self.classifier(elu_out)
        return output

In [32]:
model = BiLSTMNER(vocab_size = len(word_to_idx), target_size = len(tag_to_idx))

In [33]:
def read_in_dev_data(filename):
    sentences = []
    tmp_sentence = []
    targets = []
    tmp_target = []
    with open(filename) as file:
        for line in file.readlines():
            if len(line) > 1:
                _, word, tag = line.strip().split(" ")
                tmp_sentence.append(word)
                tmp_target.append(tag)
            else:
                sentences.append(tmp_sentence)
                targets.append(tmp_target)
                tmp_sentence = []
                tmp_target = []
        if len(tmp_sentence)>0:
            sentences.append(tmp_sentence)
            targets.append(tmp_target)
    return sentences, targets

# dev dataset

In [34]:
dev_sentences, dev_targets = read_in_dev_data("./data/dev")

In [35]:
data_X_dev = process_sentences(dev_sentences, word_to_idx)
data_y_dev = process_targets(dev_targets, tag_to_idx)

In [36]:
data_X_dev = padding_sentence(data_X_dev)
data_y_dev = padding_tags(data_y_dev)

In [37]:
X_dev = torch.LongTensor(data_X_dev)
Y_dev = torch.LongTensor(data_y_dev)
ds_dev = TensorDataset(X_dev, Y_dev)
loader_dev = DataLoader(ds_dev, batch_size=16, shuffle=False)

# test dataset

In [38]:
def read_in_test_data(filename):
    sentences = []
    tmp_sentence = []
    with open(filename) as file:
        for line in file.readlines():
            if len(line) > 1:
                _, word = line.strip().split(" ")
                tmp_sentence.append(word)
            else:
                sentences.append(tmp_sentence)
                tmp_sentence = []
        if len(tmp_sentence)>0:
            sentences.append(tmp_sentence)
    return sentences

test_sentences = read_in_test_data("./data/test")

In [39]:
data_test = process_sentences(test_sentences, word_to_idx)

In [40]:
data_test = process_sentences(test_sentences, word_to_idx)
data_X_test = padding_sentence(data_test)
X_test = torch.LongTensor(data_X_test)
loader_test = DataLoader(X_test, batch_size=16, shuffle=False)

# Using GPU

In [41]:
is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU mode")
else:
    device = torch.device("cpu")
    print("CPU mode")

GPU mode


# train and evaluate model

In [42]:
def train_evaluate(model, train_data, dev_data, epoch_num=50, tag_pad_idx = -100):
    optimizer = optim.SGD(model.parameters(), lr=0.23, momentum=0.9, nesterov=True) # Set hyperparameter
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=4)
    criterion = nn.CrossEntropyLoss(ignore_index= -100)
    best_loss = float('inf')
    predict_table = []
    for epoch in range(epoch_num):
        # training
        train_result = []
        train_loss = 0
        train_acc = 0
        train_total = 0
        model.train()
        for train_sentence, train_target in train_data:
            optimizer.zero_grad()
            train_sentence = train_sentence.to(device)
            train_target = train_target.to(device)
            train_pred = model(train_sentence)
            train_pred = train_pred.view(-1, train_pred.shape[-1])
            train_target = train_target.view(-1)
            train_tmp_loss = criterion(train_pred, train_target)

            train_tmp_total = 0
            train_tmp_correct = 0
            max_pred = train_pred.argmax(dim=1, keepdim = True)
            for tmp_pred, tmp_target, tmp_word in zip(max_pred, train_target, train_sentence.view(-1)):
                if tmp_word != 0:
                    train_result.append((tmp_word.item(), tmp_pred.item(), tmp_target.item()))
                    if tmp_target.item() == tmp_pred.item():
                        train_tmp_correct += 1
                    train_total += 1
            train_tmp_loss.backward()
            optimizer.step()
            train_loss += train_tmp_loss.item()
            train_acc += train_tmp_correct
            train_total += train_tmp_total
        print('Epoch ', epoch, ' :')
        print(f'\tTrain Loss: {train_loss/len(train_data):.6f} | Train Acc: {(train_acc/train_total)*100:.2f}%')

        # evaluating
        dev_result = []
        dev_loss = 0
        dev_acc = 0
        dev_total = 0
        model.eval()
        with torch.no_grad():
            for dev_sentence, dev_target in dev_data:
                dev_sentence = dev_sentence.to(device)
                dev_target = dev_target.to(device)
                dev_pred = model(dev_sentence)
                dev_pred = dev_pred.view(-1, dev_pred.shape[-1])
                dev_target = dev_target.view(-1)
                dev_tmp_loss = criterion(dev_pred, dev_target)

                dev_tmp_total = 0
                dev_tmp_correct = 0
                max_pred = dev_pred.argmax(dim=1, keepdim=True)
                for tmp_pred, tmp_target, tmp_word in zip(max_pred, dev_target, dev_sentence.view(-1)):
                    if tmp_word != 0:
                        dev_result.append((tmp_word.item(), tmp_pred.item(), tmp_target.item()))
                        if tmp_target.item() == tmp_pred.item():
                            dev_tmp_correct += 1
                        dev_total += 1
                dev_loss += dev_tmp_loss.item()
                dev_acc += dev_tmp_correct
                dev_total += dev_tmp_total
        print(f'\tDev Loss: {dev_loss / len(dev_data):.6f} | Dev Acc: {(dev_acc / dev_total) * 100:.2f}%')
        if dev_loss <= best_loss:
            best_loss = dev_loss
            predict_table = dev_result
            torch.save(model.state_dict(), './model/blstm2.pt')
    return predict_table

In [43]:
model.to(device)

BiLSTMNER(
  (dropout): Dropout(p=0.33, inplace=False)
  (embedding): Embedding(10732, 100, padding_idx=0)
  (lstm): LSTM(100, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=128, bias=True)
  (elu): ELU(alpha=1.0)
  (classifier): Linear(in_features=128, out_features=9, bias=True)
)

In [44]:
model.embedding.weight.data.copy_(embedding_matrix_model)

tensor([[-0.9330, -0.1493, -0.3515,  ...,  0.4095, -0.0163,  0.2232],
        [ 0.0241,  0.1957, -0.1542,  ...,  0.1727,  0.3597,  0.0295],
        [ 0.1222,  0.1316,  0.2707,  ...,  0.3569,  0.6478, -0.4779],
        ...,
        [ 0.5779,  0.1450, -0.0967,  ...,  0.1963,  0.0664,  0.3440],
        [ 0.2902,  0.8050,  0.3119,  ..., -0.3360,  0.4600, -0.1128],
        [ 0.2095,  0.0029, -0.4363,  ...,  0.3198, -0.3137,  0.1977]],
       device='cuda:0')

In [45]:
predict_table = train_evaluate(model,loader_train,loader_dev)

Epoch  0  :
	Train Loss: 0.335653 | Train Acc: 90.86%
	Dev Loss: 0.184087 | Dev Acc: 94.61%
Epoch  1  :
	Train Loss: 0.176551 | Train Acc: 94.61%
	Dev Loss: 0.133290 | Dev Acc: 95.97%
Epoch  2  :
	Train Loss: 0.133033 | Train Acc: 95.83%
	Dev Loss: 0.110509 | Dev Acc: 96.56%
Epoch  3  :
	Train Loss: 0.110988 | Train Acc: 96.44%
	Dev Loss: 0.102620 | Dev Acc: 96.72%
Epoch  4  :
	Train Loss: 0.095492 | Train Acc: 96.92%
	Dev Loss: 0.093420 | Dev Acc: 97.12%
Epoch  5  :
	Train Loss: 0.085816 | Train Acc: 97.20%
	Dev Loss: 0.091576 | Dev Acc: 97.03%
Epoch  6  :
	Train Loss: 0.073950 | Train Acc: 97.51%
	Dev Loss: 0.081089 | Dev Acc: 97.45%
Epoch  7  :
	Train Loss: 0.065665 | Train Acc: 97.79%
	Dev Loss: 0.076630 | Dev Acc: 97.53%
Epoch  8  :
	Train Loss: 0.061474 | Train Acc: 97.93%
	Dev Loss: 0.076505 | Dev Acc: 97.54%
Epoch  9  :
	Train Loss: 0.055339 | Train Acc: 98.13%
	Dev Loss: 0.076674 | Dev Acc: 97.49%
Epoch  10  :
	Train Loss: 0.051273 | Train Acc: 98.23%
	Dev Loss: 0.073294 | Dev

In [46]:
y_pred = [int(x[1]) for x in predict_table]
i=0
new_file = open('./dev2.out', "w")
with open('./data/dev', "r") as file:
    for line in file:
        if len(line) > 1:
            idx, word, tag = line.strip().split(" ")
            new_file.write(str(idx)+' '+str(word)+' '+str(idx_to_tag[y_pred[i]])+'\n')
            i+=1
        else:
            new_file.write('\n')
file.close()
new_file.close()

# test predict

In [47]:
output = []
model.to(device)
model.eval()
for test_sentence in loader_test:
    test_sentence = test_sentence.to(device)
    test_pred = model(test_sentence)
    test_pred = test_pred.view(-1, test_pred.shape[-1])
    max_pred = test_pred.argmax(dim=1, keepdim = True)
    for tmp_pred, word in zip(max_pred, test_sentence.view(-1)):
        if word != 0:
            output.append(tmp_pred.item())

In [48]:
length = 0
with open('./data/test', "r") as file:
    for line in file:
        if len(line) > 1: 
            length+=1
file.close()
print(length)
print(len(output))

46666
46666


In [49]:
i=0
new_file = open('./test2.out', "w")
with open('./data/test', "r") as file:
    for line in file:
        if len(line) > 1:
            idx, word = line.strip().split(" ")
            new_file.write(str(idx)+' '+str(word)+' '+str(idx_to_tag[output[i]])+'\n')
            i+=1
        else:
            new_file.write('\n')
file.close()
new_file.close()